# PNG Parser in pure python

[libpng file specification](http://www.libpng.org/pub/png/spec/1.2/PNG-Structure.html)

[w3c specification](https://www.w3.org/TR/2003/REC-PNG-20031110/)

In [69]:
from re import compile as re_compile
from pathlib import Path
from dataclasses import dataclass, field
from io import BytesIO
import struct
import os
# import logging

In [70]:
# settings
VERBOSE = True
VERBOSITY = 1 # 1: do not print but still write to file, 2: we ball

SAVE_PRINT_STDOUT = True
SAVE_PRINT_PRUNE = True
SAVE_PRINT_FNAME = 'stdout.txt'

# homepage http://www.schaik.com/pngsuite/
# schaik libpng webpage http://www.libpng.org/pub/png/pngsuite.html
# license http://www.schaik.com/pngsuite/PngSuite.LICENSE
STD_TEST_INPUT_TEST = True
STD_TEST_INPUT_FOLDER = 'test input'
STD_TEST_INPUT_CURL = 'http://www.schaik.com/pngsuite/PngSuite-2017jul19.zip' 
STD_TEST_INPUT_SCHAIK_EXCERPT = 'libpng-schaik-excerpt.txt'

TEST_FLUSH_TO_FS = False
TEST_FLUSH_FOLDER = 'export'

# base constants
PNG_SIGNATURE = bytearray([137, 80, 78, 71, 13, 10, 26, 10])

NULL_SEP = b'\0' # or b'\x00'

In [71]:
def _p_uint(b):
  return int.from_bytes(b, byteorder='big', signed=False)

def _p_int(b):
  return int.from_bytes(b, byteorder='big', signed=True)

In [72]:
def at_eof(fo):
    # lmao
    c = fo.read(1)
    fo.seek(-1, 1)
    if not c:
        return True
    return False

re_png_ext = re_compile(r'.*(?:\.(?:p|P)(?:n|N)(?:g|G))(?:\n|\Z)')

def is_path_png(path):
    r = re_png_ext.search(path)
    if r:
        return True
    return False

def splice_null_sep(b: bytes):
    i = b.find(NULL_SEP)
    if i == -1:
        return b, b''
    return b[:i], b[i+1:]

suppress_print_w = False

def print_w(*argv, sep=' ', end='\n', fp=SAVE_PRINT_FNAME):
    # lmfao
    if SAVE_PRINT_STDOUT:
        with open(fp, 'a') as fo:
            print(*argv, sep=sep, end=end, file=fo)
    if not suppress_print_w:
        print(*argv, sep=sep, end=end)

ls_png_test_fname_excerpt = {}

def load_png_test_fname_preset():
    if not STD_TEST_INPUT_SCHAIK_EXCERPT:
        return
    if not os.path.exists(STD_TEST_INPUT_SCHAIK_EXCERPT):
        return
    with open(STD_TEST_INPUT_SCHAIK_EXCERPT, 'r') as fo:
        for line in fo.readlines():
            if line.startswith('        '):
                r = [i.strip() for i in line.split('-')]
                if len(r) != 2:
                    continue
                kw, v = r
                ls_png_test_fname_excerpt[kw] = v

def parse_png_test_fname_preset(path: Path):
    if not STD_TEST_INPUT_TEST:
        return ''

    if not ls_png_test_fname_excerpt:
        load_png_test_fname_preset()

    if isinstance(path, Path):
        name = path.name
    elif isinstance(path, str):
        name = os.path.split(path)[-1]
    name = name.split('.')[0]

    r = ls_png_test_fname_excerpt.get(name)
    if not r:
        return 'not documented'
    return r

In [73]:
# crc implementation http://www.libpng.org/pub/png/spec/1.2/PNG-CRCAppendix.html

crc_table = [None for _ in range(256)]
is_crc_table_computed = False

def make_crc_table():
    for n in range(256):
        c = n
        for _ in range(8):
            if c & 1:
                c = 0xedb88320 ^ (c >> 1)
            else:
                c = c >> 1
        crc_table[n] = c
    is_crc_table_computed = True


def update_crc_table(crc_cksum, msg):
    c = crc_cksum

    if not is_crc_table_computed:
        make_crc_table()
    
    for b in msg:
        c = crc_table[(c ^ b) & 0xff] ^ (c >> 8)
    return c

def crc(msg: bytes):
    return update_crc_table(0xffffffff, msg) ^ 0xffffffff

def crc_to_bytes(msg: bytes):
    return struct.pack('>L', crc(msg))


In [74]:
# inits

if SAVE_PRINT_PRUNE:
    if os.path.exists(SAVE_PRINT_FNAME) and os.path.isfile(SAVE_PRINT_FNAME):
        os.remove(SAVE_PRINT_FNAME)
    with open(SAVE_PRINT_FNAME, 'w') as _:
        pass

cwd = Path('.')

if STD_TEST_INPUT_TEST:

    cwd /= STD_TEST_INPUT_FOLDER
    if not cwd.exists(): cwd.mkdir()

    if not list(cwd.glob('*.png')):
        import requests
        import zipfile

        r = requests.get(STD_TEST_INPUT_CURL)
        r.raise_for_status()
        buf = BytesIO(r.content)
        with zipfile.ZipFile(buf) as zf:
            zf.extractall(STD_TEST_INPUT_FOLDER)

target_files = list(cwd.glob('*.png'))
target_files[:10], target_files[-10:]

([WindowsPath('test input/basi0g01.png'),
  WindowsPath('test input/basi0g02.png'),
  WindowsPath('test input/basi0g04.png'),
  WindowsPath('test input/basi0g08.png'),
  WindowsPath('test input/basi0g16.png'),
  WindowsPath('test input/basi2c08.png'),
  WindowsPath('test input/basi2c16.png'),
  WindowsPath('test input/basi3p01.png'),
  WindowsPath('test input/basi3p02.png'),
  WindowsPath('test input/basi3p04.png')],
 [WindowsPath('test input/xhdn0g08.png'),
  WindowsPath('test input/xlfn0g04.png'),
  WindowsPath('test input/xs1n0g01.png'),
  WindowsPath('test input/xs2n0g01.png'),
  WindowsPath('test input/xs4n0g01.png'),
  WindowsPath('test input/xs7n0g01.png'),
  WindowsPath('test input/z00n2c08.png'),
  WindowsPath('test input/z03n2c08.png'),
  WindowsPath('test input/z06n2c08.png'),
  WindowsPath('test input/z09n2c08.png')])

In [75]:
class _BaseEnum: pass

def enum_fetch_all(enum: _BaseEnum):
  _reserved = list(dir(object)) +  ['__dict__', '__module__', '__weakref__']
  return [getattr(enum, i) for i in dir(enum) if i not in _reserved]

class ENUM_PNG_CT_BYTES(_BaseEnum):

    # Critical chunk types 
    IHDR = b'IHDR'
    PLTE = b'PLTE'
    IDAT = b'IDAT'
    IEND = b'IEND'

    # Ancillary chunk types
    cHRM = b'cHRM'    # Before PLTE and IDAT
    gAMA = b'gAMA'    # Before PLTE and IDAT
    iCCP = b'iCCP'    # Before PLTE and IDAT
    sBIT = b'sBIT'    # Before PLTE and IDAT
    sRGB = b'sRGB'    # Before PLTE and IDAT
    bKGD = b'bKGD'    # After PLTE; before IDAT
    hIST = b'hIST'    # After PLTE; before IDAT
    tRNS = b'tRNS'    # After PLTE; before IDAT
    pHYs = b'pHYs'    # Before IDAT
    sPLT = b'sPLT'    # Before IDAT
    tIME = b'tIME'
    iTXt = b'iTXt'
    tEXt = b'tEXt'
    zTXt = b'zTXt'

In [76]:
enum_fetch_all(ENUM_PNG_CT_BYTES)

[b'IDAT',
 b'IEND',
 b'IHDR',
 b'PLTE',
 b'bKGD',
 b'cHRM',
 b'gAMA',
 b'hIST',
 b'iCCP',
 b'iTXt',
 b'pHYs',
 b'sBIT',
 b'sPLT',
 b'sRGB',
 b'tEXt',
 b'tIME',
 b'tRNS',
 b'zTXt']

In [77]:
LS_CT_sRGB_INTENT_V = [
    'Perceptual',
    'Relative colorimetric',
    'Saturation',
    'Absolute colorimetric'
]

In [78]:
class PNGImage: pass

class I_ChunkTemplate: pass

class CT_IHDR(I_ChunkTemplate): pass
class CT_PLTE(I_ChunkTemplate): pass
class CT_PLTE(I_ChunkTemplate): pass
class CT_IDAT(I_ChunkTemplate): pass
class CT_IEND(I_ChunkTemplate): pass

class CT_cHRM(I_ChunkTemplate): pass
class CT_gAMA(I_ChunkTemplate): pass
class CT_iCCP(I_ChunkTemplate): pass
class CT_sBIT(I_ChunkTemplate): pass
class CT_sRGB(I_ChunkTemplate): pass
class CT_bKGD(I_ChunkTemplate): pass
class CT_hIST(I_ChunkTemplate): pass
class CT_tRNS(I_ChunkTemplate): pass
class CT_pHYs(I_ChunkTemplate): pass
class CT_sPLT(I_ChunkTemplate): pass
class CT_tIME(I_ChunkTemplate): pass
class CT_iTXt(I_ChunkTemplate): pass
class CT_tEXt(I_ChunkTemplate): pass
class CT_zTXt(I_ChunkTemplate): pass

class C_RGB: pass

In [79]:
@dataclass
class C_RGB:
    r: int = 0
    g: int = 0
    b: int = 0

In [80]:
@dataclass
class I_ChunkTemplate:
    _png_instance   : PNGImage  = None
    is_parsed       : bool      = False
    chunk_size      : int       = 0
    chunk_type      : bytes     = b''
    chunk_data      : BytesIO   = None
    chunk_crc       : bytes     = b''


    def set_pngImageInstance(self, instance):
        self._png_instance = instance

    def _parser(self):
        print_w(f'WARNING! chunk type {self.chunk_type} parser method is not yet overridden. skipping procedure.')
        return 1

    def _test(self):
        # print_w(f'chunk {self.chunk_type} test method is not yet overridden.')
        return

    def parse(self):
        if self.is_parsed:
            return
        r = self._parser()
        if r:
            return
        self._test()
        self.is_parsed = True

In [81]:
@dataclass
class CT_IHDR(I_ChunkTemplate):
    width               : int = None
    height              : int = None
    bit_depth           : int = None
    color_type          : int = None
    compression_method  : int = None
    filter_method       : int = None
    interlace_method    : int = None
    color_type_info     : str = ''

    def _parser(self):
        self.width              = _p_uint(self.chunk_data.read(4))
        self.height             = _p_uint(self.chunk_data.read(4))
        self.bit_depth          = _p_int(self.chunk_data.read(1))
        self.color_type         = _p_int(self.chunk_data.read(1))
        self.compression_method = _p_int(self.chunk_data.read(1))
        self.filter_method      = _p_int(self.chunk_data.read(1))
        self.interlace_method   = _p_int(self.chunk_data.read(1))
        if self.color_type & 1:
            self.color_type_info += 'palette-used;'
        if self.color_type & 2:
            self.color_type_info += 'color-used;'
        if self.color_type & 4:
            self.color_type_info += 'alpha-channel-used;'

        if self.color_type_info:
            self.color_type_info = self.color_type_info[:-1]

    def _test(self):
        assert self.width != 0, 'invalid value: width is zero'
        assert self.height != 0, 'invalid value: height is zero'

        color_type_msg = f'invalid value: color type {self.color_type} and/or bit depth {self.bit_depth} does not match specification'

        # assert self.color_type in [0, 1, 2, 3, 4, 6], color_type_msg

        # if self.color_type == 0:
        #     assert self.bit_depth in [1, 2, 4, 8, 16], color_type_msg

        # elif self.color_type == 3:
        #     assert self.bit_depth in [1, 2, 4, 8], color_type_msg

        # elif self.color_type == 2 \
        #     or self.color_type == 4 \
        #     or self.color_type == 6:
        #     assert self.bit_depth in [8, 16], color_type_msg


@dataclass
class CT_PLTE(I_ChunkTemplate):
    entries : list = field(default_factory=list)
    
    def _parser(self):
        assert self.chunk_size % 3 == 0, 'invalid value: chunk length is not divisible by 3'
        l = self.chunk_size // 3
        for _ in range(l):
            c_rgb = C_RGB()
            c_rgb.r = _p_uint(self.chunk_data.read(1))
            c_rgb.g = _p_uint(self.chunk_data.read(1))
            c_rgb.b = _p_uint(self.chunk_data.read(1))

            self.entries.append(c_rgb)
    
@dataclass
class CT_IDAT(I_ChunkTemplate):

    def _parser(self):
        pass

@dataclass
class CT_IEND(I_ChunkTemplate):

    def _parser(self):
        pass

@dataclass
class CT_cHRM(I_ChunkTemplate):
    white_point_x   : int = 0
    white_point_y   : int = 0
    red_x           : int = 0
    red_y           : int = 0
    green_x         : int = 0
    green_y         : int = 0
    blue_x          : int = 0
    blue_y          : int = 0

    def _parser(self):
        self.white_point_x = _p_uint(self.chunk_data.read(4))
        self.white_point_y = _p_uint(self.chunk_data.read(4))
        self.red_x         = _p_uint(self.chunk_data.read(4))
        self.red_y         = _p_uint(self.chunk_data.read(4))
        self.green_x       = _p_uint(self.chunk_data.read(4))
        self.green_y       = _p_uint(self.chunk_data.read(4))
        self.blue_x        = _p_uint(self.chunk_data.read(4))
        self.blue_y        = _p_uint(self.chunk_data.read(4))

@dataclass
class CT_gAMA(I_ChunkTemplate):
    gamma: int = 0

    def _parser(self):
        self.gamma = _p_uint(self.chunk_data.read(4))

@dataclass
class CT_iCCP(I_ChunkTemplate):
    profile_name        : bytes = None
    compression_method  : bytes = None
    compression_profile : bytes = None

    def _parser(self):
        chunk = self.chunk_data.read()
        chunk_split = chunk.split(NULL_SEP)

        self.profile_name, compression_info = chunk_split
        self.compression_method = compression_info[0:1]
        self.compression_profile = compression_info[1:]


@dataclass
class CT_sBIT(I_ChunkTemplate):
    ...

@dataclass
class CT_sRGB(I_ChunkTemplate):
    rendering_intent : int = None
    rendering_intent_value : str = None
    
    def _parser(self):
      self.rendering_intent = _p_uint(self.chunk_data.read(1))
      self.rendering_intent_value = LS_CT_sRGB_INTENT_V[self.rendering_intent]

@dataclass
class CT_bKGD(I_ChunkTemplate):
    background_color : C_RGB = C_RGB()

    def _parser(self):

        ct_IHDR = self._png_instance.get_chunk(ENUM_PNG_CT_BYTES.IHDR, do_parse=True)
        color_type = ct_IHDR.color_type

        if color_type == 3:
            ct_PLTE = self._png_instance.get_chunk(ENUM_PNG_CT_BYTES.PLTE, do_parse=True)
            i = _p_uint(self.chunk_data.read(1))
            c = ct_PLTE.entries[i]
            self.background_color.r = c.r
            self.background_color.g = c.g
            self.background_color.b = c.b
        elif color_type == 0 or color_type == 4:
            c = _p_uint(self.chunk_data.read(2))
            self.background_color.r = c
            self.background_color.g = c
            self.background_color.b = c
        elif color_type == 2 or color_type == 6:
            self.background_color.r = _p_uint(self.chunk_data.read(2))
            self.background_color.g = _p_uint(self.chunk_data.read(2))
            self.background_color.b = _p_uint(self.chunk_data.read(2))

@dataclass
class CT_hIST(I_ChunkTemplate):
    ...

@dataclass
class CT_tRNS(I_ChunkTemplate):
    alpha_channel : list = field(default_factory=list)

    def _parser(self):
        ct_IHDR = self._png_instance.get_chunk(ENUM_PNG_CT_BYTES.IHDR, do_parse=True)
        bit_depth = ct_IHDR.bit_depth
        color_type = ct_IHDR.color_type

        if color_type == 0:
            self.alpha_channel.append(
                _p_uint(self.chunk_data.read(2))
            )

        if color_type == 2:
            c_rgb = C_RGB()
            c_rgb.r = _p_uint(self.chunk_data.read(2))
            c_rgb.g = _p_uint(self.chunk_data.read(2))
            c_rgb.b = _p_uint(self.chunk_data.read(2))
            self.alpha_channel.append(
                c_rgb
            )

        if color_type == 3:
            for _ in range(self.chunk_size):
                self.alpha_channel.append(
                    _p_uint(self.chunk_data.read(1))
                )

    def _test(self):
        ct_IHDR = self._png_instance.get_chunk(ENUM_PNG_CT_BYTES.IHDR)
        color_type = ct_IHDR.color_type
        assert color_type not in [4, 6], f'invalid value: {self.chunk_type} prohibited for color types 4 and 6'

@dataclass
class CT_pHYs(I_ChunkTemplate):
    pixel_per_unit_x : int = None
    pixel_per_unit_y : int = None
    unit_specifier   : int = None

    def _parser(self):
        self.pixel_per_unit_x = _p_uint(self.chunk_data.read(4))
        self.pixel_per_unit_y = _p_uint(self.chunk_data.read(4))
        self.unit_specifier = _p_uint(self.chunk_data.read(1))

    def _test(self):
        assert self.unit_specifier in [0, 1]

@dataclass
class CT_sPLT(I_ChunkTemplate):
    ...

@dataclass
class CT_tIME(I_ChunkTemplate):
    year    : int = None
    month   : int = None
    day     : int = None
    hour    : int = None
    minute  : int = None
    second  : int = None

    def _parser(self):
        self.year   = _p_uint(self.chunk_data.read(2))
        self.month  = _p_uint(self.chunk_data.read(1))
        self.day    = _p_uint(self.chunk_data.read(1))
        self.hour   = _p_uint(self.chunk_data.read(1))
        self.minute = _p_uint(self.chunk_data.read(1))
        self.second = _p_uint(self.chunk_data.read(1))

    def _test(self):
        assert self.month  in range(1, 13)
        assert self.day    in range(1, 32)
        assert self.hour   in range(24)
        assert self.minute in range(60)
        assert self.second in range(61)

@dataclass
class CT_iTXt(I_ChunkTemplate):
    keyword             : bytes = None
    compression_flag    : int = None
    compression_method  : int = None
    language_tag        : bytes = None
    translated_keyword  : bytes = None
    text                : bytes = None

    def _parser(self):
        chunk = self.chunk_data.read()

        self.keyword, chunk = splice_null_sep(chunk)
        self.compression_flag, chunk = chunk[:1], chunk[1:]
        self.compression_method, chunk = chunk[:1], chunk[1:]
        self.language_tag, chunk = splice_null_sep(chunk)
        self.translated_keyword, chunk = splice_null_sep(chunk)
        self.text, chunk = splice_null_sep(chunk)

@dataclass
class CT_tEXt(I_ChunkTemplate):
    keyword : bytes = ''
    text    : bytes = ''
    
    def _parser(self):
        chunk = self.chunk_data.read()
        self.keyword, self.text = splice_null_sep(chunk)


@dataclass
class CT_zTXt(I_ChunkTemplate):
    keyword             : bytes = None
    compression_method  : int = None
    compressed_text     : bytes = None

    def _parser(self):
        chunk = self.chunk_data.read()
        self.keyword, chunk = splice_null_sep(chunk)
        self.compression_method, self.compressed_text = chunk[:1], chunk[1:]


In [82]:
class PNGImage:
    verbose : bool = False

    def __init__(self):
        self.signature = PNG_SIGNATURE
        self._chunks   = list()
        self._fname    = ''
        self._ext      = '.png'

    def set_filename(self, filename: str):
        self._fname = filename.split('.')[0]

    def get_filename(self):
        return self._fname + self._ext

    def set_signature(self, signature: bytes):
        self.signature = signature

    def check_signature(self, signature: bytes):
        return signature == PNG_SIGNATURE

    def append_chunk(self, ct: I_ChunkTemplate):
        ct.set_pngImageInstance(self)
        self._chunks.append(ct)

    def iter_chunks(self):
        for i in self._chunks:
            yield i

    def get_chunks(self, chunk_type_enum = None, do_parse = False):
        if chunk_type_enum:
            r = list(filter(lambda x: x.chunk_type == chunk_type_enum, self._chunks))
        else:
            r = self._chunks

        if do_parse:
            _ = [i.parse() for i in r]

        return r


    def get_chunk(self, chunk_type_enum = None, do_parse = False):
        r = self.get_chunks(chunk_type_enum)[0]
        if do_parse:
            r.parse()
        return r

    def parse_all(self):
        for c in self._chunks:
            if self.verbose: print_w(f'parsing {c.chunk_type}; {c.chunk_size} bytes')
            c.parse()

    def flush(self, fo):
        fo.write(self.signature)
        for c in self.iter_chunks():
            chunk_data = b''
            if c.chunk_data:
                c.chunk_data.seek(0)
                chunk_data = c.chunk_data.read()
            fo.write(struct.pack('>I', c.chunk_size))
            fo.write(c.chunk_type)
            fo.write(chunk_data)
            fo.write(crc_to_bytes(c.chunk_type + chunk_data))

    def __repr__(self):

        # default:
        #
        # return '<%s.%s object at %s>' % (
        #     self.__class__.__module__,
        #     self.__class__.__name__,
        #     hex(id(self))
        # )

        return '<%s>' % (
            self.__class__.__name__,
        )

PNGImage.verbose = VERBOSE

In [83]:
chunk_struct_constructor_dict = {
    ENUM_PNG_CT_BYTES.IHDR: CT_IHDR,
    ENUM_PNG_CT_BYTES.IDAT: CT_IDAT,
    ENUM_PNG_CT_BYTES.IEND: CT_IEND,
    ENUM_PNG_CT_BYTES.PLTE: CT_PLTE,

    ENUM_PNG_CT_BYTES.cHRM: CT_cHRM,
    ENUM_PNG_CT_BYTES.gAMA: CT_gAMA,
    ENUM_PNG_CT_BYTES.iCCP: CT_iCCP,
    ENUM_PNG_CT_BYTES.sBIT: CT_sBIT,
    ENUM_PNG_CT_BYTES.sRGB: CT_sRGB,
    ENUM_PNG_CT_BYTES.bKGD: CT_bKGD,
    ENUM_PNG_CT_BYTES.hIST: CT_hIST,
    ENUM_PNG_CT_BYTES.tRNS: CT_tRNS,
    ENUM_PNG_CT_BYTES.pHYs: CT_pHYs,
    ENUM_PNG_CT_BYTES.sPLT: CT_sPLT,
    ENUM_PNG_CT_BYTES.tIME: CT_tIME,
    ENUM_PNG_CT_BYTES.iTXt: CT_iTXt,
    ENUM_PNG_CT_BYTES.tEXt: CT_tEXt,
    ENUM_PNG_CT_BYTES.zTXt: CT_zTXt,
}

In [84]:
def parse_png_file(path: Path, do_parse: bool = True) -> PNGImage:

    filename = ''
    if isinstance(path, Path):
        filename = path.name
    elif isinstance(path, str):
        filename = os.path.split(path)[-1]

    if not is_path_png(filename):
        print_w(f'file {filename} does not have correct png extension')
        return

    png = PNGImage()
    png.set_filename(filename)

    with open(path, 'rb') as fo:
        png_signature = fo.read(8)
        if not png.check_signature(png_signature):
            print_w(f'WARNING! file does not contain or has correct PNG signature: {png_signature}') 
            png.set_signature(png_signature)

        while not at_eof(fo):
            chunk_size = _p_uint(fo.read(4))
            chunk_type = fo.read(4)
            chunk_data = fo.read(chunk_size)
            chunk_crc  = fo.read(4)

            if chunk_crc != crc_to_bytes(chunk_type + chunk_data):
                print_w('WARNING! file is corrupted: crc hash does not match')

            if chunk_type not in enum_fetch_all(ENUM_PNG_CT_BYTES):
                print_w(f'WARNING! chunk type {chunk_type} is not present in ENUM_PNG_CT_BYTES')
                continue

            elif chunk_type not in chunk_struct_constructor_dict:
                print_w(f'WARNING! chunk type {chunk_type} parser is not yet implemented')
                continue

            t = chunk_struct_constructor_dict.get(chunk_type)()
            t.chunk_size = chunk_size
            t.chunk_data = BytesIO(chunk_data) if chunk_size else None
            t.chunk_type = chunk_type
            t.chunk_crc = chunk_crc

            png.append_chunk(t)

        if do_parse:
            png.parse_all()

    return png


In [85]:
# %%capture parse_pngs --no-display

for target_file_path in target_files:

    suppress_print_w = True
    if VERBOSE: print_w('-------------------------------------------------')
    suppress_print_w = False
    if VERBOSE: print_w('target file      :', f'"{target_file_path}"')
    if STD_TEST_INPUT_TEST and VERBOSE:
        print_w('test description :', parse_png_test_fname_preset(target_file_path))
    suppress_print_w = True
    if VERBOSE: print_w('-------------------------------------------------')


    png = parse_png_file(target_file_path)

    if VERBOSE: print_w(f'parsed chunks:', )
    if png:
        for n, c in enumerate(png.iter_chunks()):
            if VERBOSE: print_w(f'  [{n:03}] -', c)
    
        if TEST_FLUSH_TO_FS:
            with open(Path(TEST_FLUSH_FOLDER) / png.get_filename(), 'wb') as fo:
                png.flush(fo)


    if VERBOSE: print_w()


target file      : "test input\basi0g01.png"
test description : black & white
target file      : "test input\basi0g02.png"
test description : 2 bit (4 level) grayscale
target file      : "test input\basi0g04.png"
test description : 4 bit (16 level) grayscale
target file      : "test input\basi0g08.png"
test description : 8 bit (256 level) grayscale
target file      : "test input\basi0g16.png"
test description : 16 bit (64k level) grayscale
target file      : "test input\basi2c08.png"
test description : 3x8 bits rgb color
target file      : "test input\basi2c16.png"
test description : 3x16 bits rgb color
target file      : "test input\basi3p01.png"
test description : 1 bit (2 color) paletted
target file      : "test input\basi3p02.png"
test description : 2 bit (4 color) paletted
target file      : "test input\basi3p04.png"
test description : 4 bit (16 color) paletted
target file      : "test input\basi3p08.png"
test description : 8 bit (256 color) paletted
target file      : "test input